<a href="https://www.kaggle.com/code/msrahman7/smart-blood-donor-matcher-raktabondhu-agent?scriptVersionId=282635586" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 1. Uninstall conflicting packages to avoid "Model not found" errors
!pip uninstall -y google-cloud-aiplatform bigframes

# 2. Install the latest required libraries
!pip install -U langchain-google-genai langchain langchain-community gradio google-generativeai pandas

In [1]:
# app.py — Smart Blood Donor Matcher (Raktabondhu)
# Final — single-file stable release (validate UX: keep inputs, highlight faulty field labels)

import os
import re
import time
import uuid
import shutil
from datetime import datetime, timedelta, date
import pandas as pd
import gradio as gr
from tempfile import NamedTemporaryFile
import threading

# Optional LangChain (kept optional)
try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain.agents import initialize_agent, AgentType
    from langchain.tools import tool
    LANGCHAIN_AVAILABLE = True
except Exception:
    LANGCHAIN_AVAILABLE = False

# -----------------------
# CONFIG
# -----------------------
ADMIN_PASSWORD = os.environ.get("ADMIN_PASSWORD", "admin123")
DB_FILE = "blood_donors.csv"
CONTACT_FILE = "contact_requests.csv"
TEMP_EXPORT_DIR = "/tmp/raktabondhu_exports"
BACKUP_DIR = "backups"
EXPORT_EXPIRE_SECONDS = 300

os.makedirs(TEMP_EXPORT_DIR, exist_ok=True)
os.makedirs(BACKUP_DIR, exist_ok=True)

# -----------------------
# Helpers: atomic write + backup
# -----------------------
def atomic_write_df_to_csv(df: pd.DataFrame, path: str):
    dirp = os.path.dirname(os.path.abspath(path)) or "."
    with NamedTemporaryFile("w", delete=False, dir=dirp, suffix=".tmp", newline="") as tf:
        tmpname = tf.name
        df.to_csv(tmpname, index=False)
    os.replace(tmpname, path)

def backup_file(path: str):
    try:
        if os.path.exists(path):
            ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
            base = os.path.basename(path)
            bname = f"{base}.{ts}.bak"
            dst = os.path.join(BACKUP_DIR, bname)
            shutil.copy2(path, dst)
    except Exception:
        pass

# -----------------------
# Date component fallback
# -----------------------
def DateComponent(label, value=None, placeholder="YYYY-MM-DD"):
    # uses gr.Date when available, otherwise Textbox placeholder
    if hasattr(gr, "Date"):
        return gr.Date(label=label, value=value)
    else:
        initial = value
        if isinstance(value, (datetime, date)):
            initial = value.strftime("%Y-%m-%d")
        return gr.Textbox(label=label, value=initial if initial is not None else "", placeholder=placeholder)

# -----------------------
# DB Schema and init
# -----------------------
REQUIRED_COLUMNS = [
    "ID", "Name", "Phone", "Blood_Group", "Location", "Status",
    "Registered_At", "Last_Donation_Date", "Next_Eligible_Date"
]

CONTACT_COLUMNS = ["Req_ID", "Requested_At", "Donor_ID_or_Phone", "Requester_Phone", "Message", "Approved", "Reviewed_At"]

def init_db():
    if not os.path.exists(DB_FILE):
        df = pd.DataFrame(columns=REQUIRED_COLUMNS)
        atomic_write_df_to_csv(df, DB_FILE)
    else:
        try:
            df = pd.read_csv(DB_FILE, dtype=str)
        except Exception:
            backup_file(DB_FILE)
            df = pd.DataFrame(columns=REQUIRED_COLUMNS)
            atomic_write_df_to_csv(df, DB_FILE)
            return
        changed = False
        for col in REQUIRED_COLUMNS:
            if col not in df.columns:
                if col == "ID":
                    df["ID"] = [str(uuid.uuid4()) for _ in range(len(df))]
                else:
                    df[col] = ""
                changed = True
        if changed:
            backup_file(DB_FILE)
            atomic_write_df_to_csv(df, DB_FILE)

def init_contact_log():
    if not os.path.exists(CONTACT_FILE):
        df = pd.DataFrame(columns=CONTACT_COLUMNS)
        atomic_write_df_to_csv(df, CONTACT_FILE)
    else:
        try:
            df = pd.read_csv(CONTACT_FILE, dtype=str)
        except Exception:
            backup_file(CONTACT_FILE)
            df = pd.DataFrame(columns=CONTACT_COLUMNS)
            atomic_write_df_to_csv(df, CONTACT_FILE)
            return
        changed = False
        for col in CONTACT_COLUMNS:
            if col not in df.columns:
                df[col] = ""
                changed = True
        if changed:
            backup_file(CONTACT_FILE)
            atomic_write_df_to_csv(df, CONTACT_FILE)

init_db()
init_contact_log()

# -----------------------
# Utility functions
# -----------------------
def sanitize_for_csv_cell(val):
    if pd.isna(val):
        return ""
    s = str(val).strip()
    s = s.replace("\r", " ").replace("\n", " ")
    if s and s[0] in ("=", "+", "-", "@"):
        return "'" + s
    return s

def mask_phone(p):
    p = str(p or "")
    digits = re.sub(r"\D", "", p)
    if len(digits) <= 4:
        return "****"
    return digits[:3] + "*" * (max(1, len(digits) - 5)) + digits[-2:]

def short_name(n):
    n = str(n or "").strip()
    if not n:
        return ""
    parts = n.split()
    if len(parts) == 1:
        return parts[0]
    if len(parts) == 2:
        return parts[0] + " " + parts[1][0] + "."
    return parts[0] + " " + parts[1][0] + "."

from datetime import datetime as _datetime, date as _date
def parse_date_safe(s):
    if s is None:
        return None
    if isinstance(s, _date) and not isinstance(s, _datetime):
        return _datetime(s.year, s.month, s.day)
    if isinstance(s, _datetime):
        return s
    s = str(s).strip()
    if not s:
        return None
    for fmt in ("%Y-%m-%d", "%Y-%m-%d %H:%M:%S", "%d-%m-%Y", "%d/%m/%Y"):
        try:
            return _datetime.strptime(s, fmt)
        except Exception:
            continue
    try:
        return pd.to_datetime(s)
    except Exception:
        return None

def next_eligible_from(last_dt):
    if not last_dt:
        return ""
    if isinstance(last_dt, str):
        last_dt = parse_date_safe(last_dt)
    if last_dt is None:
        return ""
    next_dt = last_dt + timedelta(days=90)
    return next_dt.strftime("%Y-%m-%d")

# -----------------------
# FIX: Consistent Phone Normalization (Problem 1)
# -----------------------
def normalize_phone_for_compare(phone: str) -> str:
    if not phone:
        return ""
    digits = re.sub(r"\D", "", str(phone))
    
    # Remove leading '88' if present and the length is > 10 (e.g. 88017...)
    if digits.startswith("88") and len(digits) > 10:
        digits = digits[2:]
        
    # Take the last 10 digits for comparison (e.g. 017xxxxxxx -> 17xxxxxxx)
    if len(digits) >= 10:
        digits = digits[-10:]
        
    # Return the last 10 digits as a standardized comparison key
    return digits if len(digits) == 10 else "" 

def validate_phone(phone: str) -> bool:
    if not phone:
        return False
    digits = re.sub(r"\D", "", str(phone))
    return 9 < len(digits) <= 13

def clear_form_fields():
    """Returns updates to clear all input fields on the Registration tab."""
    return (
        gr.update(value=""),      # agent_response_top (Clear message)
        gr.update(value=""),      # name_input
        gr.update(value=""),      # phone_input
        gr.update(value=""),      # phone_error_html
        gr.update(value=None),    # bg_input
        gr.update(value=""),      # loc_input
        gr.update(value="Available"), # status_input (Reset to default)
        gr.update(value=""),      # last_donation_input
        gr.update(label="Full Name *"), # Reset name label
        gr.update(label="Phone Number *"), # Reset phone label
        gr.update(label="Blood Group *"), # Reset BG label
    )

# -----------------------
# Helper: check if phone already registered
# -----------------------
def phone_exists(phone: str) -> bool:
    if not phone:
        return False
    # Use the normalized form for searching
    norm_query = normalize_phone_for_compare(phone)
    if not norm_query:
        return False
        
    df = read_donors_df(mask=False)
    if df.empty:
        return False
        
    for _, r in df.iterrows():
        # Compare normalized query against normalized stored phone
        if normalize_phone_for_compare(r.get("Phone", "")) == norm_query:
            return True
            
    return False

# -----------------------
# Read/write DB helpers
# -----------------------
def read_donors_df(mask=True):
    if not os.path.exists(DB_FILE):
        init_db()
    try:
        df = pd.read_csv(DB_FILE, dtype=str)
    except Exception:
        backup_file(DB_FILE)
        df = pd.DataFrame(columns=REQUIRED_COLUMNS)
        atomic_write_df_to_csv(df, DB_FILE)
        return df if not mask else df
    if df is None:
        df = pd.DataFrame(columns=REQUIRED_COLUMNS)
    if not df.empty:
        try:
            df = df.sort_values(by="Registered_At", ascending=False).reset_index(drop=True)
        except Exception:
            df = df.reset_index(drop=True)
    if not df.empty:
        if "Next_Eligible_Date" not in df.columns:
            df["Next_Eligible_Date"] = ""
        changed = False
        for i, row in df.iterrows():
            if not row.get("Next_Eligible_Date"):
                df.at[i, "Next_Eligible_Date"] = next_eligible_from(row.get("Last_Donation_Date"))
                changed = True
        if changed:
            backup_file(DB_FILE)
            atomic_write_df_to_csv(df, DB_FILE)
    if not df.empty:
        df = df.fillna("").astype(str)
    if mask and not df.empty:
        dfm = df.copy()
        dfm["Name"] = dfm["Name"].apply(short_name)
        dfm["Phone"] = dfm["Phone"].apply(mask_phone)
        dfm["Location"] = dfm["Location"].apply(lambda x: str(x).split(",")[0].strip() if x else "")
        return dfm
    return df

def save_new_donor(name, phone, bg, loc, status, last_donation_date=None):
    name_s = sanitize_for_csv_cell(name)
    phone_s = sanitize_for_csv_cell(phone)
    bg_s = sanitize_for_csv_cell(bg)
    loc_s = sanitize_for_csv_cell(loc)
    status_s = sanitize_for_csv_cell(status)
    last_s = ""
    next_s = ""
    if last_donation_date:
        dt = parse_date_safe(last_donation_date)
        if dt:
            last_s = dt.strftime("%Y-%m-%d")
            next_s = next_eligible_from(dt)
    ts = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
    df = read_donors_df(mask=False)
    new_id = str(uuid.uuid4())
    new_entry = pd.DataFrame([[new_id, name_s, phone_s, bg_s, loc_s, status_s, ts, last_s, next_s]],
                             columns=REQUIRED_COLUMNS)
    df = pd.concat([df, new_entry], ignore_index=True)
    backup_file(DB_FILE)
    atomic_write_df_to_csv(df, DB_FILE)

# -----------------------
# Choices & helpers
# -----------------------
def extract_id_from_choice(selected_choice: str) -> str:
    if not selected_choice:
        return ""
    parts = str(selected_choice).rsplit(" - ", 1)
    return parts[-1] if parts else ""

def donor_choices(mask=True):
    df = read_donors_df(mask=mask)
    choices = []
    if df.empty:
        return choices
    for _, row in df.reset_index().iterrows():
        id_ = row.get("ID", "")
        phone_display = row.get("Phone", "")
        name = row.get("Name", "")
        bg = row.get("Blood_Group", "")
        choices.append(f"{phone_display} - {short_name(name)} ({bg}) - {id_}")
    return choices

# -----------------------
# Improved find-matches function
# -----------------------
def find_matches_by_fragment(fragment: str, max_results=50):
    if not fragment:
        return "<div style='padding:8px;'>Enter some digits or an ID fragment to find matches.</div>"
    frag_raw = str(fragment).strip()
    frag_digits = re.sub(r"\D", "", frag_raw)
    df = read_donors_df(mask=False)
    if df.empty:
        return "<div style='padding:8px;'>No donors in database.</div>"

    results = []

    # 1) ID exact or substring
    if frag_raw:
        for _, r in df.iterrows():
            if frag_raw in str(r.get("ID", "")):
                results.append(r)

    # 2) try matching normalized last-10 exact
    if not results and frag_digits:
        frag_last10 = frag_digits[-10:]
        for _, r in df.iterrows():
            stored_norm = normalize_phone_for_compare(r.get("Phone", ""))
            if stored_norm and stored_norm == frag_last10:
                results.append(r)

    # 3) try endswith / contains on stored normalized phone
    if not results and frag_digits:
        for _, r in df.iterrows():
            stored_norm = normalize_phone_for_compare(r.get("Phone", ""))
            if stored_norm and (stored_norm.endswith(frag_digits) or frag_digits in stored_norm):
                results.append(r)

    # 4) masked-like pattern e.g. 019******96 — improved: use regex allowing gaps where * are
    if not results and "*" in frag_raw:
        frag_pattern = "".join(ch for ch in frag_raw if ch.isdigit() or ch == "*")
        if frag_pattern:
            regex_pattern = frag_pattern.replace("*", r"\d*")
            prog = re.compile(regex_pattern)
            for _, r in df.iterrows():
                stored_digits = re.sub(r"\D", "", str(r.get("Phone", "")))
                if stored_digits and prog.search(stored_digits):
                    results.append(r)

    if not results:
        return "<div style='padding:8px;'>No matches found for that fragment.</div>"

    html = "<div style='padding:8px;'><strong>Matches:</strong><ul>"
    for r in results[:max_results]:
        html += ("<li style='margin-bottom:6px;'>"
                 f"<strong>{sanitize_for_csv_cell(r.get('Name',''))}</strong> &nbsp;|&nbsp; "
                 f"{mask_phone(r.get('Phone',''))} &nbsp;|&nbsp; {sanitize_for_csv_cell(r.get('Location',''))} &nbsp;|&nbsp; "
                 f"{sanitize_for_csv_cell(r.get('Blood_Group',''))} &nbsp;|&nbsp; ID: <code>{r.get('ID')}</code>"
                 "</li>")
    html += "</ul></div>"
    return html

# -----------------------
# Report donation
# -----------------------
def report_donation(phone, donation_date):
    if not phone:
        return ("Error: Please provide your phone number.",
                read_donors_df(mask=True), read_donors_df(mask=False))
    if not validate_phone(phone):
        return ("Error: Invalid phone format.",
                read_donors_df(mask=True), read_donors_df(mask=False))
    dt = parse_date_safe(donation_date)
    if dt is None:
        return ("Error: Donation date not recognized. Use YYYY-MM-DD.",
                read_donors_df(mask=True), read_donors_df(mask=False))
    digits = normalize_phone_for_compare(phone)
    df = read_donors_df(mask=False)
    found_idx = None
    for i, row in df.iterrows():
        if normalize_phone_for_compare(row.get("Phone", "")) == digits:
            found_idx = i
            break
    if found_idx is None:
        return ("Error: Donor with that phone not found. Please register first.",
                read_donors_df(mask=True), read_donors_df(mask=False))
    df.at[found_idx, "Last_Donation_Date"] = dt.strftime("%Y-%m-%d")
    df.at[found_idx, "Next_Eligible_Date"] = next_eligible_from(dt)
    backup_file(DB_FILE)
    atomic_write_df_to_csv(df, DB_FILE)
    name = df.at[found_idx, "Name"]
    return (f"Success: Thank you {name}. Last donation date updated to {dt.strftime('%Y-%m-%d')}.",
            read_donors_df(mask=True), read_donors_df(mask=False))

# -----------------------
# Contact requests (with Req_ID)
# -----------------------
def make_req_id():
    ts = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    short = uuid.uuid4().hex[:6]
    return f"REQ-{ts}-{short}"

def request_contact(donor_id_or_phone, requester_phone, message):
    if not requester_phone or not validate_phone(requester_phone):
        return ("Error: Provide a valid phone for requester.", get_contact_requests())
    ts = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
    reqid = make_req_id()
    row = {"Req_ID": reqid, "Requested_At": ts, "Donor_ID_or_Phone": sanitize_for_csv_cell(donor_id_or_phone),
           "Requester_Phone": sanitize_for_csv_cell(requester_phone), "Message": sanitize_for_csv_cell(message),
           "Approved": "", "Reviewed_At": ""}
    if os.path.exists(CONTACT_FILE):
        df = pd.read_csv(CONTACT_FILE, dtype=str)
    else:
        df = pd.DataFrame(columns=CONTACT_COLUMNS)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    backup_file(CONTACT_FILE)
    atomic_write_df_to_csv(df, CONTACT_FILE)
    return (f"Request submitted (ID: {reqid}). Admin will review and respond.", get_contact_requests())

def get_contact_requests():
    if not os.path.exists(CONTACT_FILE):
        init_contact_log()
    df = pd.read_csv(CONTACT_FILE, dtype=str)
    if df is None:
        return pd.DataFrame(columns=CONTACT_COLUMNS)
    for c in CONTACT_COLUMNS:
        if c not in df.columns:
            df[c] = ""
    df = df[CONTACT_COLUMNS]
    return df.fillna("")

def find_contact_index_by_reqid(reqid: str):
    if not reqid:
        return None
    df = get_contact_requests()
    matches = df.index[df["Req_ID"] == reqid].tolist()
    if not matches:
        return None
    return int(matches[0])

def approve_contact_request_by_index(idx):
    df = get_contact_requests()
    try:
        idx = int(idx)
        if idx < 0 or idx >= len(df):
            return ("Error: Invalid request index.", get_contact_requests())
        df.at[idx, "Approved"] = "YES"
        df.at[idx, "Reviewed_At"] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
        backup_file(CONTACT_FILE)
        atomic_write_df_to_csv(df, CONTACT_FILE)
        return ("Approved.", get_contact_requests())
    except Exception as e:
        return (f"Error: {e}", get_contact_requests())

def reject_contact_request_by_index(idx):
    df = get_contact_requests()
    try:
        idx = int(idx)
        if idx < 0 or idx >= len(df):
            return ("Error: Invalid request index.", get_contact_requests())
        df.at[idx, "Approved"] = "NO"
        df.at[idx, "Reviewed_At"] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
        backup_file(CONTACT_FILE)
        atomic_write_df_to_csv(df, CONTACT_FILE)
        return ("Rejected.", get_contact_requests())
    except Exception as e:
        return (f"Error: {e}", get_contact_requests())

def approve_contact_request_by_reqid(reqid):
    idx = find_contact_index_by_reqid(reqid)
    if idx is None:
        return ("Error: Req ID not found.", get_contact_requests())
    return approve_contact_request_by_index(idx)

def reject_contact_request_by_reqid(reqid):
    idx = find_contact_index_by_reqid(reqid)
    if idx is None:
        return ("Error: Req ID not found.", get_contact_requests())
    return reject_contact_request_by_index(idx)

def approve_wrapper(admin_ok, idx, reqid):
    if not admin_ok:
        return ("Error: Admin required.", get_contact_requests(), gr.update(value=None), gr.update(value=""))
    if reqid and str(reqid).strip():
        msg, df = approve_contact_request_by_reqid(reqid.strip())
    else:
        msg, df = approve_contact_request_by_index(idx)
    return (msg, df, gr.update(value=None), gr.update(value=""))

def reject_wrapper(admin_ok, idx, reqid):
    if not admin_ok:
        return ("Error: Admin required.", get_contact_requests(), gr.update(value=None), gr.update(value=""))
    if reqid and str(reqid).strip():
        msg, df = reject_contact_request_by_reqid(reqid.strip())
    else:
        msg, df = reject_contact_request_by_index(idx)
    return (msg, df, gr.update(value=None), gr.update(value=""))

def delete_all_contact_requests(admin_ok):
    if not admin_ok:
        return ("Error: Admin required.", get_contact_requests())
    try:
        empty = pd.DataFrame(columns=CONTACT_COLUMNS)
        backup_file(CONTACT_FILE)
        atomic_write_df_to_csv(empty, CONTACT_FILE)
        return ("All contact requests deleted.", get_contact_requests())
    except Exception as e:
        return (f"Error deleting requests: {e}", get_contact_requests())

# -----------------------
# Admin delete donor by ID
# -----------------------
def delete_donor_by_id(admin_ok, donor_id):
    if not admin_ok:
        return ("Error: Admin required.", read_donors_df(mask=True), gr.update(choices=donor_choices(mask=True)))
    if not donor_id or not str(donor_id).strip():
        return ("Error: Provide ID to delete.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
    try:
        df = read_donors_df(mask=False)
        donor_id = str(donor_id).strip()
        if donor_id not in df["ID"].values:
            return ("Error: ID not found.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
        df = df[df["ID"] != donor_id].reset_index(drop=True)
        backup_file(DB_FILE)
        atomic_write_df_to_csv(df, DB_FILE)
        return ("Success: Deleted donor.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
    except Exception as e:
        return (f"System Error: {e}", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))

# -----------------------
# Delete contact request by Req ID
# -----------------------
def delete_request_by_reqid(admin_ok, reqid):
    if not admin_ok:
        return ("Error: Admin required.", get_contact_requests())
    if not reqid or not str(reqid).strip():
        return ("Error: Provide Req ID to delete.", get_contact_requests())
    try:
        df = get_contact_requests()
        reqid = str(reqid).strip()
        matches = df.index[df["Req_ID"] == reqid].tolist()
        if not matches:
            return ("Error: Req ID not found.", get_contact_requests())
        idx = matches[0]
        df = df.drop(idx).reset_index(drop=True)
        backup_file(CONTACT_FILE)
        atomic_write_df_to_csv(df, CONTACT_FILE)
        return ("Success: Deleted contact request.", get_contact_requests())
    except Exception as e:
        return (f"Error deleting request: {e}", get_contact_requests())

# -----------------------
# Wrapper: delete by ID or by REQ-... (admin only)
# -----------------------
def delete_by_id_wrapper(admin_ok, id_value):
    if not admin_ok:
        return ("Error: Admin required.", read_donors_df(mask=False), get_contact_requests(), gr.update(choices=donor_choices(mask=False)))

    val = (id_value or "").strip()
    if not val:
        return ("Error: Provide an ID.", read_donors_df(mask=False), get_contact_requests(), gr.update(choices=donor_choices(mask=False)))

    try:
        if val.upper().startswith("REQ-"):
            msg, df_req = delete_request_by_reqid(True, val)
            return (msg, read_donors_df(mask=False), df_req, gr.update(choices=donor_choices(mask=False)))
        else:
            msg, donors_df, select_update = delete_donor_by_id(True, val)
            return (msg, donors_df, get_contact_requests(), select_update)
    except Exception as e:
        return (f"Error in delete wrapper: {e}", read_donors_df(mask=False), get_contact_requests(), gr.update(choices=donor_choices(mask=False)))

# -----------------------
# Phone search for Register page (Find my record)
# -----------------------
def find_by_phone(phone):
    if not phone:
        return ("Error: Enter phone to search.", "", "", None, "", "Available", "")
    raw = str(phone).strip()
    norm_input_digits = re.sub(r"\D", "", raw)
    if not norm_input_digits:
        return ("Error: Invalid phone format.", "", "", None, "", "Available", "")

    df = read_donors_df(mask=False)
    if df.empty:
        return ("No record found.", "", "", None, "", "Available", "")

    norm_last10 = normalize_phone_for_compare(norm_input_digits)
    for _, r in df.iterrows():
        stored_norm = normalize_phone_for_compare(r.get("Phone", ""))
        if stored_norm and stored_norm == norm_last10:
            return ("Found", r.get("Name", ""), r.get("Phone", ""), r.get("Blood_Group", ""),
                    r.get("Location", ""), r.get("Status", "Available"), r.get("Last_Donation_Date", ""))

    # fragment
    frag = norm_last10
    if frag:
        for _, r in df.iterrows():
            stored_norm = normalize_phone_for_compare(r.get("Phone", ""))
            if stored_norm and frag in stored_norm:
                return ("Found (by fragment)", r.get("Name", ""), r.get("Phone", ""), r.get("Blood_Group", ""),
                        r.get("Location", ""), r.get("Status", "Available"), r.get("Last_Donation_Date", ""))

    digits_only = re.sub(r"\D", "", raw)
    if digits_only and len(digits_only) >= 3:
        for _, r in df.iterrows():
            if digits_only in re.sub(r"\D", "", str(r.get("Phone", ""))):
                return ("Found (by fragment)", r.get("Name", ""), r.get("Phone", ""), r.get("Blood_Group", ""),
                        r.get("Location", ""), r.get("Status", "Available"), r.get("Last_Donation_Date", ""))

    return ("No record found.", "", "", None, "", "Available", "")

# -----------------------
# Agent / Chat helpers
# -----------------------
BLOOD_GROUPS = {"a+","a-","b+","b-","o+","o-","ab+","ab-"}

def extract_bg_and_location(text):
    if not text:
        return (None, None)
    t = str(text).lower()
    bg_found = None
    for g in BLOOD_GROUPS:
        if g in t:
            bg_found = g.upper()
            break
    loc = None
    m = re.search(r"(?:in|near|at)\s+([a-zA-Z\u0980-\u09FF0-9\s\-\_]+)", t)
    if m:
        loc = m.group(1).strip().rstrip(".")
    else:
        parts = [p.strip() for p in re.split(r"[,\n]", text) if p.strip()]
        if len(parts) >= 2:
            cand = parts[-1]
            if len(cand.split()) <= 6:
                loc = cand
    return (bg_found, loc)

def find_donors_local(bg=None, loc=None, max_rows=10):
    df = read_donors_df(mask=False)
    if df.empty:
        return df.iloc[0:0]
    if bg:
        df = df[df['Blood_Group'].str.lower() == bg.lower()]
    if loc:
        df = df[df['Location'].str.lower().str.contains(str(loc).lower(), na=False)]
    return df.head(max_rows)

def format_search_results(df):
    if df is None or df.empty:
        return "No donors found for that query."
    lines = []
    for _, r in df.iterrows():
        lines.append(f"- {r['Name']} | {mask_phone(r['Phone'])} | {r['Location']} | {r['Blood_Group']} | {r.get('Status','')}")
    return "Found:\n" + "\n".join(lines)

def process_chat(message, history):
    message = (message or "").strip()
    if not message:
        return "Please type your request (e.g. 'Find A+ donor in Dhaka')."
    if LANGCHAIN_AVAILABLE and 'agent' in globals():
        try:
            out = agent.run(message)
            if out:
                return out
        except Exception:
            pass
    bg, loc = extract_bg_and_location(message)
    if not bg and not loc:
        return ("I couldn't detect blood group or location.\nExamples:\n- Find A+ donor in Dhaka\n- O- donor near Chittagong\n\nTo get full contact details you must request contact from Admin (use the 'Contact Request' tab).")
    df = find_donors_local(bg=bg, loc=loc, max_rows=25)
    return format_search_results(df)

# -----------------------
# Minimal CSS (light)
# -----------------------
CSS = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700;800&display=swap');

* { font-family: 'Inter', sans-serif; box-sizing: border-box; color:#0b1220; }

.form-card {
    max-width: 1100px;
    margin: 18px auto;
    padding: 20px;
    border-radius: 12px;
    background: linear-gradient(180deg, #ffffff, #fbfbfd);
    box-shadow: 0 10px 30px rgba(11,18,32,0.06);
    border:1px solid rgba(15,23,42,0.04);
}

.top-title { display:flex; align-items:center; gap:12px; margin: 6px auto; justify-content:center; }
.top-title h1 { margin:0; font-size:20px; font-weight:700; letter-spacing:0.2px; }
.top-sub { margin-top:6px; font-size:13px; color:#6b7280; text-align:center; margin-bottom:14px; }

.input-row { display:flex; gap:12px; margin-bottom:12px; }
.input-row > .gr-block { flex:1; }

@media (max-width:900px) { .input-row { flex-direction:column; } }

.submit-btn button {
    border-radius:10px !important;
    padding:9px 14px !important;
    font-weight:700 !important;
    cursor:pointer;
    background: linear-gradient(90deg,#ff7a00,#ff9100) !important;
    color:white !important;
    width:100% !important;
    box-shadow: 0 8px 20px rgba(255,121,0,0.12);
}
.small-btn button {
    background:#f3f4f6 !important;
    color:#0b1220 !important;
    border-radius:8px !important;
    padding:8px 12px !important;
    font-weight:600 !important;
}

.gr-dataframe { max-height: 320px; overflow:auto; border-radius:8px; border:1px solid rgba(15,23,42,0.03); }

.agent-response-box { background: linear-gradient(180deg,#f0f9ff,#eef8ff); border: 1px solid #cfeeff; padding:10px 12px; border-radius:8px; color:#073b6f; margin-bottom:12px; font-weight:600; }

.admin-section { background:#ffffff; border-radius:10px; padding:12px; border:1px solid rgba(15,23,42,0.04); margin-bottom:12px; box-shadow:0 6px 18px rgba(11,18,32,0.03); }
.admin-section .section-title { font-weight:700; font-size:14px; margin-bottom:10px; color:#0b1220; }

.center-btn button { width:160px !important; margin:6px auto !important; display:block !important; text-align:center !important; padding:8px 12px !important; border-radius:8px !important; background: linear-gradient(90deg,#0ea5a4,#06b6d4) !important; color:white !important; font-weight:700 !important; box-shadow:none !important; }
.center-btn.reject button { background: linear-gradient(90deg,#ef4444,#f97316) !important; }

#phone-input + .gr-html { margin-top:6px; margin-bottom:6px; } /* phone error container */
"""

# -----------------------
# Form processing (validate + save)
# -----------------------
def process_form(name, phone, bg, loc, status, last_donation_date):
    """
    Return tuple matching outputs expected by UI wiring:
    (message_plain, donors_table_public_df,
     name_input_update, phone_input_update, phone_error_html_update,
     bg_input_update, loc_input_update, status_input_update, last_donation_input_update,
     donors_table_admin_df, select_dd_update)
    - On validation error: keep field values (do NOT clear). Update only the offending label/html.
    - On success: save donor, clear form fields and clear phone_error_html.
    """
    try:
        donors_public = read_donors_df(mask=True)
        donors_admin = read_donors_df(mask=False)

        no_change = gr.update()
        clear_phone_error = gr.update(value="")

        # Name required
        if not name or str(name).strip() == "":
            name_label_update = gr.update(label="Full Name * — REQUIRED")
            return (
                "Error: Full Name is required.",
                donors_public,
                name_label_update,
                no_change,
                clear_phone_error,
                no_change, no_change, no_change, no_change,
                donors_admin,
                gr.update(choices=donor_choices(mask=False))
            )

        # Phone presence
        if not phone or str(phone).strip() == "":
            phone_label_update = gr.update(label="Phone Number * — REQUIRED")
            return (
                "Error: Phone number is required.",
                donors_public,
                no_change,
                phone_label_update,
                clear_phone_error,
                no_change, no_change, no_change, no_change,
                donors_admin,
                gr.update(choices=donor_choices(mask=False))
            )

        # Phone format
        if not validate_phone(phone):
            phone_label_update = gr.update(label="Phone Number * — INVALID FORMAT")
            phone_error_html = gr.update(value="<div style='color:#b91c1c; font-weight:600;'>Invalid phone format. Use e.g. 01700000000</div>")
            return (
                "Error: Invalid phone format.",
                donors_public,
                no_change,
                phone_label_update,
                phone_error_html,
                no_change, no_change, no_change, no_change,
                donors_admin,
                gr.update(choices=donor_choices(mask=False))
            )

        # Blood group required
        if not bg or str(bg).strip() == "":
            bg_label_update = gr.update(label="Blood Group * — REQUIRED")
            return (
                "Error: Blood Group is required.",
                donors_public,
                no_change,
                no_change,
                clear_phone_error,
                bg_label_update,
                no_change, no_change, no_change,
                donors_admin,
                gr.update(choices=donor_choices(mask=False))
            )

        
        # Duplicate phone check — DO NOT show bottom big message; instead change phone label to include red ALREADY REGISTERED
        if phone_exists(phone):
            # label with inline red span (Original label restored)
            phone_label_update = gr.update(label="Phone Number *") 
            
            # Show inline phone error HTML below the phone field
            phone_error_html = gr.update(value="<div style='color:#b91c1c; font-weight:700;'>ALREADY REGISTERED (Use Find My Record to check)</div>")
            
            # Use the top HTML box for the main error message, styled in red.
            # This output is used by the wrapper function process_form_wrapped_top to display the error.
            # Note: We return the text/message here, and the wrapper formats it to HTML.
            
            return (
                "Error: This phone number is already registered.", # <-- This will be wrapped in HTML by process_form_wrapped_top
                donors_public,
                no_change,
                phone_label_update,
                phone_error_html,
                no_change, no_change, no_change, no_change,
                donors_admin,
                gr.update(choices=donor_choices(mask=False))
            )

        # Passed validation: save
        save_new_donor(name, phone, bg, loc, status or "Available", last_donation_date)
        success_msg = f"Success: Registered {short_name(name)}."

        return (
            success_msg,
            read_donors_df(mask=True),
            gr.update(value=""),
            gr.update(value=""),
            gr.update(value=""),
            gr.update(value=None),
            gr.update(value=""),
            gr.update(value="Available"),
            gr.update(value=""),
            read_donors_df(mask=False),
            gr.update(choices=donor_choices(mask=False))
        )

    except Exception as e:
        return (
            f"Error saving donor: {e}",
            read_donors_df(mask=True),
            gr.update(), gr.update(), gr.update(value=""),
            gr.update(), gr.update(), gr.update(), gr.update(),
            read_donors_df(mask=False),
            gr.update(choices=donor_choices(mask=False))
        )

def process_form_wrapped_top(name, phone, bg, loc, status, last_donation_date):
    """
    Wrap process_form to return a styled top-box message (HTML) while preserving
    the exact tuple shape expected by the UI wiring.
    """
    out = process_form(name, phone, bg, loc, status, last_donation_date)
    msg = out[0] if isinstance(out, tuple) and len(out) else str(out)
    styled = f"<div class='agent-response-box'>{sanitize_for_csv_cell(msg)}</div>"
    new_out = list(out)
    new_out[0] = styled
    return tuple(new_out)

# -----------------------
# Admin Specific Functions
# -----------------------

def admin_find_by_phone(phone_query):
    # 1. Input Validation and Normalization
    if not phone_query or not str(phone_query).strip():
        all_choices_full = donor_choices(mask=False)
        return (gr.update(choices=all_choices_full, value=None), 
                gr.update(value=f"Please enter a query. Showing all {len(all_choices_full)} donors.", visible=True))
    
    q_raw = str(phone_query).strip()
    q_digits = re.sub(r"\D", "", q_raw)
    
    all_choices = donor_choices(mask=False)
    matched = []
    
    # Normalize query to the standardized 10-digit format
    q_norm = normalize_phone_for_compare(q_raw)
    
    # Priority 1: Normalized Phone Match (most reliable)
    if q_norm:
        # Create a list of donor IDs that match the normalized phone
        df_unmasked = read_donors_df(mask=False)
        
        # Ensure 'Phone' column exists and is treated as string
        if 'Phone' in df_unmasked.columns:
            df_unmasked['Phone_Norm'] = df_unmasked['Phone'].apply(normalize_phone_for_compare)
            
            # Find donors where the normalized phone matches the normalized query
            matching_ids = df_unmasked[df_unmasked['Phone_Norm'] == q_norm]['ID'].tolist()
            
            # Filter the 'all_choices' list to include only matching donor IDs
            for ch in all_choices:
                 if ch.rsplit(' - ', 1)[-1] in matching_ids:
                    if ch not in matched: matched.append(ch)
        
    # Priority 2: Fuzzy Name/ID Match (If no strict phone match found)
    if not matched:
        for ch in all_choices:
            # Check for raw string match on any part of the choice string (case-insensitive)
            if q_raw.lower() in ch.lower():
                if ch not in matched: matched.append(ch)

    
    # Final Output Handling
    if not matched:
        # No match found, return the full list and an explicit error status
        all_choices_full = donor_choices(mask=False)
        return (gr.update(choices=all_choices_full, value=None), 
                gr.update(value=f"Error: No match found for '{q_raw}'. Showing all {len(all_choices_full)} donors.", visible=True))
    else:
        # Matches found, return only the matched items
        return (gr.update(choices=matched, value=matched[0] if matched else None), 
                gr.update(value=f"Found {len(matched)} match(es). Select donor from dropdown.", visible=True))


# -----------------------
# UI
# -----------------------
with gr.Blocks(title="Smart Blood Donor Matcher (Raktabondhu)") as demo:
    gr.HTML(f"<style>{CSS}</style>")
    gr.HTML("<div class='top-title'><span style='font-size:26px;'>🩸</span><h1>Smart Blood Donor Matcher (Raktabondhu)</h1></div>")
    gr.HTML("<div class='top-sub'>Use the Chat to find donors or the Register tab to add new donors.</div>")

    with gr.Column(elem_classes="form-card"):
        with gr.Tabs():
            # Chat Tab
            with gr.Tab("💬 Chat & Search (Enhanced)"):
                with gr.Row():
                    with gr.Column(scale=3):
                        gr.Markdown("### 🗣️ Chatbot\nType your message or use quick chips.")
                        gr.HTML("<div class='agent-response-box'><strong>Note:</strong> After finding donors, to obtain full contact details you must request contact via the 'Contact Request' tab. Admin will review your request and respond.</div>")
                    with gr.Column(scale=1):
                        with gr.Accordion("How to use / Tips", open=False):
                            gr.Markdown("**Examples:**\n- Find A+ donor in Dhaka\n- O- donor near Chittagong")
                chatbot = gr.Chatbot(label="Chatbot", height=340, type="messages")
                chat_input = gr.Textbox(placeholder="Type message (e.g. Find A+ donor in Dhaka)...", elem_id="chat-input")
                send_btn = gr.Button("Send")
                with gr.Row():
                    qA = gr.Button("A+"); qB = gr.Button("B+"); qO = gr.Button("O+"); qAB = gr.Button("AB+")
                chat_state = gr.State([])

                def chat_submit(message, history):
                    history = history or []
                    msg = (message or "").strip()
                    if not msg:
                        return history, "", history
                    history.append({"role":"user","content": msg})
                    try:
                        reply_text = process_chat(msg, history)
                    except Exception as e:
                        reply_text = f"System error: {e}"
                    history.append({"role":"assistant","content": reply_text})
                    return history, "", history

                send_btn.click(fn=chat_submit, inputs=[chat_input, chat_state], outputs=[chatbot, chat_input, chat_state])
                chat_input.submit(fn=chat_submit, inputs=[chat_input, chat_state], outputs=[chatbot, chat_input, chat_state])

                def quick_chip(bg_label, history):
                    return chat_submit(f"Find {bg_label} donor", history)

                qA.click(fn=quick_chip, inputs=[gr.State("A+"), chat_state], outputs=[chatbot, chat_input, chat_state])
                qB.click(fn=quick_chip, inputs=[gr.State("B+"), chat_state], outputs=[chatbot, chat_input, chat_state])
                qO.click(fn=quick_chip, inputs=[gr.State("O+"), chat_state], outputs=[chatbot, chat_input, chat_state])
                qAB.click(fn=quick_chip, inputs=[gr.State("AB+"), chat_state], outputs=[chatbot, chat_input, chat_state])

            # Register Tab
            with gr.Tab("📝 Register (Public)"):
                agent_response_top = gr.HTML("", elem_id="agent-response-top")
                gr.Markdown("### New Donor Registration (Public)")

                # Row: name + phone (REQUIRED fields)
                with gr.Row(elem_classes="input-row"):
                    name_input = gr.Textbox(label="Full Name *", placeholder="e.g. Shohidur Rahman")
                    phone_input = gr.Textbox(label="Phone Number *", placeholder="e.g. 01700000000", elem_id="phone-input")

                # inline phone error area (kept but by default empty)
                phone_error_html = gr.HTML("", elem_id="phone-error")

                # Row: blood group + location (Blood Group = REQUIRED)
                with gr.Row(elem_classes="input-row"):
                    bg_input = gr.Dropdown(choices=["A+","A-","B+","B-","O+","O-","AB+","AB-"], label="Blood Group *", allow_custom_value=True)
                    loc_input = gr.Textbox(label="Location (City/Area)", placeholder="e.g. Ashuganj")

                # Row: last donation + status
                with gr.Row(elem_classes="input-row"):
                    last_donation_input = DateComponent(label="Last Donation Date (optional)")
                    status_input = gr.Dropdown(choices=["Available","Unavailable"], label="Status", value="Available", allow_custom_value=True)

                # Submit / Find my record / Clear buttons (Duplicate row removed, single correct row retained)
                with gr.Row():
                    submit_btn = gr.Button("Submit Registration", elem_classes="submit-btn")
                    find_btn = gr.Button("Find My Record")
                    clear_btn = gr.Button("Clear Form", elem_classes="small-btn") # Added Clear button

                # Report Donation area
                gr.Markdown("### Report a Recent Donation")
                with gr.Row(elem_classes="input-row"):
                    report_phone = gr.Textbox(label="Your Phone Number", placeholder="e.g. 01700000000")
                    report_date = DateComponent(label="Donation Date", value=date.today())
                report_btn = gr.Button("Report Donation")
                report_msg = gr.Textbox(label="Report Status")

                # Find matches / lookup
                gr.Markdown("#### Find donor phone/ID (optional)")
                contact_lookup = gr.Textbox(label="Donor phone or ID (optional)", placeholder="Type full phone, last digits, or ID fragment")
                find_matches_btn = gr.Button("Find matches")
                find_matches_html = gr.HTML("", elem_id="find-matches-html")

                # Public donors table (masked)
                donors_table_public = gr.Dataframe(value=read_donors_df(mask=True),
                                                     headers=["ID","Name","Phone","Blood_Group","Location","Status","Registered_At","Last_Donation_Date","Next_Eligible_Date"],
                                                     label="Donors (public view - masked)")
                gr.Markdown("**Public view is masked.**")

            # Contact Request Tab
            with gr.Tab("📨 Contact Request"):
                gr.Markdown("### Request Contact (Admin will review)\nFill donor phone or ID (optional), your phone, and a short message. Admin will review and contact if approved.")
                with gr.Row(elem_classes="input-row"):
                    contact_lookup_tab = gr.Textbox(label="Donor phone or ID (optional)", placeholder="019xxxxxxx or donor ID")
                    requester_phone = gr.Textbox(label="Your Phone (for reply)", placeholder="e.g. 017xxxxxxxx")
                contact_message = gr.Textbox(label="Message (optional)", placeholder="Short reason for contact")
                contact_btn = gr.Button("Request Contact", elem_id="request-contact-btn")
                contact_msg = gr.Textbox(label="Request Status", elem_id="request-status")

            # Admin Tab
            with gr.Tab("🔒 Admin"):
                gr.Markdown("### Admin Access (Login required)")
                
                # --- START: Login Control Area (Fixed Layout) ---
                with gr.Row(elem_classes="input-row"):
                    # Column 1: Password Input, Login/Logout Buttons
                    with gr.Column(scale=3):
                        admin_pass = gr.Textbox(label="Admin Password", type="password", placeholder="Enter admin password")
                        with gr.Row():
                            admin_login_btn = gr.Button("🔑 Login", elem_classes="submit-btn") # Added emoji
                            logout_btn = gr.Button("🚪 Logout", elem_classes="small-btn") # Added emoji
                            
                    # Column 2: Status Display
                    with gr.Column(scale=1):
                        admin_status = gr.Textbox(label="Admin Status", interactive=False, max_lines=1)
                # --- END: Login Control Area ---

                # --- Ping (Debug) Section (Visible before login - Problem 2 Fix) ---
                gr.Markdown("<div class='section-sep'></div>")
                test_btn = gr.Button("Ping (debug)")
                test_out = gr.Textbox(label="Ping result", interactive=False)
                test_btn.click(fn=lambda: "OK from server", inputs=None, outputs=[test_out])
                gr.Markdown("<div class='section-sep'></div>")
                # --- End Ping Section ---

                admin_block = gr.Column(visible=False)
                with admin_block:
                    gr.Markdown("<div class='admin-section'><div class='section-title'>Admin Panel (Full Data & Controls)</div></div>", elem_id="admin-top")

                    # --- START: Donors Table (Full Width - Problem 1 Fix) ---
                    # Donors Table now occupies the entire space
                    donors_table_admin = gr.Dataframe(value=read_donors_df(mask=False),
                                                         headers=["ID","Name","Phone","Blood_Group","Location","Status","Registered_At","Last_Donation_Date","Next_Eligible_Date"],
                                                         label="Donors (Admin full view)")
                    # --- END: Donors Table ---

                    gr.Markdown("<div class='section-sep'></div>")
                    gr.Markdown("### 🔍 Quick Search, Lookup & Edit Control") # New separated section header

                    # --- START: Search and Dropdown (Now below the table) ---
                    with gr.Row():
                        with gr.Column(scale=3):
                            # Search Inputs
                            admin_phone_search = gr.Textbox(label="Search by Phone (any format)")
                            admin_search_btn = gr.Button("Search", elem_classes="small-btn")
                            admin_message_search = gr.Textbox(label="Search Status", interactive=False, visible=False)
                        
                        with gr.Column(scale=1):
                            # Dropdown for selecting the found donor
                            select_dd = gr.Dropdown(choices=donor_choices(mask=False), label="Select Donor to Edit/Delete", allow_custom_value=True)
                    # --- END: Search and Dropdown ---

                    gr.Markdown("---")

                    # Edit Fields Area
                    with gr.Row(elem_classes="input-row"):
                        edit_name = gr.Textbox(label="Edit Name")
                        edit_phone = gr.Textbox(label="Edit Phone")
                    with gr.Row(elem_classes="input-row"):
                        edit_bg = gr.Dropdown(choices=["A+","A-","B+","B-","O+","O-","AB+","AB-"], label="Edit Blood Group", allow_custom_value=True)
                        edit_loc = gr.Textbox(label="Edit Location")
                    with gr.Row(elem_classes="input-row"):
                        edit_last = DateComponent(label="Edit Last Donation Date")
                        edit_status = gr.Dropdown(choices=["Available","Unavailable"], label="Edit Status", allow_custom_value=True)
                    with gr.Row():
                        edit_btn = gr.Button("Save Changes", elem_classes="small-btn")
                        delete_btn = gr.Button("Delete Donor", elem_classes="small-btn")
                    admin_message = gr.Textbox(label="Admin Message")
                    with gr.Row():
                        export_btn = gr.Button("Export CSV", elem_classes="small-btn")
                        csv_file = gr.File(label="Download CSV (temp)")

                    gr.Markdown("<div class='section-sep'></div>")
                    gr.Markdown("<div class='admin-section'><div class='section-title'>Contact Requests</div>")
                    contact_requests_df = gr.Dataframe(value=get_contact_requests(), label="Contact Requests (Admin view)")
                    gr.Markdown("</div>")

                    # Approve / Reject area in two columns
                    gr.Markdown("<div style='margin-top:10px;'></div>")
                    with gr.Row():
                        with gr.Column(scale=1):
                            gr.Markdown("<div class='admin-section'><div class='small-label'>Approve by index or Req ID</div>")
                            approve_idx = gr.Number(label="Index (0-based)", value=None)
                            approve_reqid = gr.Textbox(label="Or paste Req ID to Approve", placeholder="REQ-2025... (optional)")
                            approve_btn = gr.Button("Approve", elem_classes="center-btn")
                            gr.Markdown("</div>")
                        with gr.Column(scale=1):
                            gr.Markdown("<div class='admin-section'><div class='small-label'>Reject by index or Req ID</div>")
                            reject_idx = gr.Number(label="Index (0-based)", value=None)
                            reject_reqid = gr.Textbox(label="Or paste Req ID to Reject", placeholder="REQ-2025... (optional)")
                            reject_btn = gr.Button("Reject", elem_classes="center-btn reject")
                            gr.Markdown("</div>")

                    gr.Markdown("<div class='section-sep'></div>")

                    with gr.Row():
                        with gr.Column():
                            delete_all_btn = gr.Button("Delete All Requests", elem_classes="small-btn")
                            gr.Markdown("*(Deletes all contact requests — admin only)*")
                        with gr.Column():
                            delete_by_id_input = gr.Textbox(label="Delete donor by ID (paste ID here)")
                            delete_by_id_btn = gr.Button("Delete by ID", elem_classes="small-btn")

    # -----------------------
    # Wiring / callbacks
    # -----------------------

    # Register Tab wiring (submit)
    submit_btn.click(
        fn=process_form_wrapped_top,
        inputs=[name_input, phone_input, bg_input, loc_input, status_input, last_donation_input],
        outputs=[
            agent_response_top,     # styled HTML message
            donors_table_public,    # refreshed public table
            name_input,             # update name_input (clear on success / label-change on error)
            phone_input,            # update phone_input (clear on success / label-change on error)
            phone_error_html,       # inline phone error HTML (red text) — used only for specific format errors
            bg_input,               # update bg_input
            loc_input,              # update loc_input
            status_input,           # update status_input
            last_donation_input,    # update last_donation_input
            donors_table_admin,     # refreshed admin table
            select_dd               # update admin select dropdown choices
        ]
    )
    
    # Clear Form wiring
    clear_btn.click(fn=clear_form_fields, inputs=None,
                     outputs=[
                         agent_response_top, 
                         name_input, 
                         phone_input, 
                         phone_error_html,
                         bg_input, 
                         loc_input, 
                         status_input, 
                         last_donation_input,
                         name_input, # Resetting label for name
                         phone_input, # Resetting label for phone
                         bg_input # Resetting label for BG
                     ]
                 )


    find_btn.click(fn=find_by_phone,
                     inputs=[phone_input],
                     outputs=[agent_response_top, name_input, phone_input, bg_input, loc_input, status_input, last_donation_input])

    report_btn.click(fn=report_donation, inputs=[report_phone, report_date],
                      outputs=[report_msg, donors_table_public, donors_table_admin])

    find_matches_btn.click(fn=find_matches_by_fragment, inputs=[contact_lookup], outputs=[find_matches_html])

    # Contact request
    def request_contact_wrapped(donor, requester, msg):
        out_msg, df = request_contact(donor, requester, msg)
        return out_msg, df

    contact_btn.click(fn=request_contact_wrapped, inputs=[contact_lookup_tab, requester_phone, contact_message],
                      outputs=[contact_msg, contact_requests_df])

    
    def admin_login(password):
        # expected outputs: (admin_status_str, admin_state_bool, donors_table_admin_df, admin_block_update, select_dd_update)
        try:
            pwd = (password or "").strip()
            if pwd == ADMIN_PASSWORD:
                return (
                    "Admin: Logged in.",
                    True,
                    read_donors_df(mask=False),
                    gr.update(visible=True),
                    gr.update(choices=donor_choices(mask=False))
                )
            else:
                return (
                    "Login failed: Incorrect password.",
                    False,
                    read_donors_df(mask=False),
                    gr.update(visible=False),
                    gr.update(choices=donor_choices(mask=False))
                )
        except Exception as e:
            return (
                f"Login error: {e}",
                False,
                read_donors_df(mask=False),
                gr.update(visible=False),
                gr.update(choices=donor_choices(mask=False))
            )


    def admin_logout():
        # expected outputs: (admin_status_str, admin_state_bool, donors_table_admin_df, admin_block_update, select_dd_update)
        try:
            return (
                "Admin: Logged out",
                False,
                read_donors_df(mask=False),
                gr.update(visible=False),
                gr.update(choices=donor_choices(mask=False))
            )
        except Exception as e:
            return (
                f"Logout error: {e}",
                False,
                read_donors_df(mask=False),
                gr.update(visible=False),
                gr.update(choices=donor_choices(mask=False))
            )

    
    
    # Admin login/logout wiring
    admin_state = gr.State(False)
    admin_login_btn.click(fn=admin_login, inputs=[admin_pass],
                          outputs=[admin_status, admin_state, donors_table_admin, admin_block, select_dd])
    logout_btn.click(fn=admin_logout, inputs=None, outputs=[admin_status, admin_state, donors_table_admin, admin_block, select_dd])

    # Admin search wiring (Problem 1 Fix connection)
    admin_search_btn.click(fn=admin_find_by_phone, 
                           inputs=[admin_phone_search], 
                           outputs=[select_dd, admin_message_search])

    # Populate edit fields
    def populate_for_edit(selected_choice):
        if not selected_choice:
            return "", "", "", None, "", "", ""
        donor_id = extract_id_from_choice(selected_choice)
        df = read_donors_df(mask=False)
        row = df[df["ID"] == donor_id]
        if row.empty:
            return ("Selected donor not found.", "", "", None, "", "", "")
        r = row.iloc[0]
        return ("", r.get("Name",""), r.get("Phone",""), r.get("Blood_Group",""), r.get("Location",""), r.get("Status",""), r.get("Last_Donation_Date",""))

    select_dd.change(fn=populate_for_edit, inputs=[select_dd],
                      outputs=[admin_message, edit_name, edit_phone, edit_bg, edit_loc, edit_status, edit_last])

    # Edit donor
    def edit_donor(admin_ok, selected_choice, name, phone, bg, loc, status, last_date):
        if not admin_ok:
            return ("Error: Admin required.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
        try:
            if not selected_choice:
                return ("Error: Select a donor to edit.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
            donor_id = extract_id_from_choice(selected_choice)
            df = read_donors_df(mask=False)
            idxs = df.index[df["ID"] == donor_id].tolist()
            if not idxs:
                return ("Error: Donor not found.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
            idx = idxs[0]
            if name is not None and str(name).strip() != "":
                df.at[idx, "Name"] = sanitize_for_csv_cell(name)
            if phone is not None and str(phone).strip() != "":
                df.at[idx, "Phone"] = sanitize_for_csv_cell(phone)
            if bg is not None and str(bg).strip() != "":
                df.at[idx, "Blood_Group"] = sanitize_for_csv_cell(bg)
            if loc is not None:
                df.at[idx, "Location"] = sanitize_for_csv_cell(loc)
            if status is not None and str(status).strip() != "":
                df.at[idx, "Status"] = sanitize_for_csv_cell(status)
            if last_date:
                dt = parse_date_safe(last_date)
                if dt:
                    df.at[idx, "Last_Donation_Date"] = dt.strftime("%Y-%m-%d")
                    df.at[idx, "Next_Eligible_Date"] = next_eligible_from(dt)
            backup_file(DB_FILE)
            atomic_write_df_to_csv(df, DB_FILE)
            return ("Saved changes.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
        except Exception as e:
            return (f"Error editing donor: {e}", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))

    edit_btn.click(fn=edit_donor,
                    inputs=[admin_state, select_dd, edit_name, edit_phone, edit_bg, edit_loc, edit_status, edit_last],
                    outputs=[admin_message, donors_table_admin, select_dd])

    # Delete donor via dropdown
    def delete_donor(admin_ok, selected_choice):
        if not admin_ok:
            return ("Error: Admin required.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
        try:
            if not selected_choice:
                return ("Error: Select a donor to delete.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
            donor_id = extract_id_from_choice(selected_choice)
            df = read_donors_df(mask=False)
            if donor_id not in df["ID"].values:
                return ("Error: Donor ID not found.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
            df = df[df["ID"] != donor_id].reset_index(drop=True)
            backup_file(DB_FILE)
            atomic_write_df_to_csv(df, DB_FILE)
            return ("Deleted donor.", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))
        except Exception as e:
            return (f"Error deleting donor: {e}", read_donors_df(mask=False), gr.update(choices=donor_choices(mask=False)))

    delete_btn.click(fn=delete_donor, inputs=[admin_state, select_dd],
                      outputs=[admin_message, donors_table_admin, select_dd])

    # Delete by ID wrapper
    delete_by_id_btn.click(fn=delete_by_id_wrapper, inputs=[admin_state, delete_by_id_input],
                            outputs=[admin_message, donors_table_admin, contact_requests_df, select_dd])

    # Export CSV
    def export_csv(admin_ok):
        if not admin_ok:
            return (None, "Error: Admin required.")
        try:
            df = read_donors_df(mask=False)
            if df.empty:
                return (None, "No data to export.")
            fname = f"raktabondhu_export_{int(time.time())}.csv"
            path = os.path.join(TEMP_EXPORT_DIR, fname)
            df.to_csv(path, index=False)
            def cleanup(p, delay):
                time.sleep(delay)
                try:
                    if os.path.exists(p):
                        os.remove(p)
                except Exception:
                    pass
            threading.Thread(target=cleanup, args=(path, EXPORT_EXPIRE_SECONDS), daemon=True).start()
            return (path, f"Export prepared: {fname}")
        except Exception as e:
            return (None, f"Export error: {e}")

    export_btn.click(fn=export_csv, inputs=[admin_state], outputs=[csv_file, admin_message])

    # Approve / Reject / Delete all wiring
    approve_btn.click(fn=approve_wrapper, inputs=[admin_state, approve_idx, approve_reqid],
                      outputs=[admin_message, contact_requests_df, approve_idx, approve_reqid])
    reject_btn.click(fn=reject_wrapper, inputs=[admin_state, reject_idx, reject_reqid],
                      outputs=[admin_message, contact_requests_df, reject_idx, reject_reqid])

    delete_all_btn.click(fn=delete_all_contact_requests, inputs=[admin_state], outputs=[admin_message, contact_requests_df])

# -----------------------
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c0fd3d5b0731618636.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c0fd3d5b0731618636.gradio.live
